<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Tools for Monitoring Backends and Jobs

In this section, we will learn how to monitor the status of jobs submitted to devices and simulators (collectively called backends), as well as discover how to easily query backend details and view the collective state of all the backends available to you.

## Loading the Monitoring Tools

First, let us load the default qiskit routines, and register our IBMQ credentials.

In [1]:
from qiskit import *
IBMQ.load_accounts()

Functions for monitoring jobs and backends are here:

In [2]:
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

## Tracking Job Status

Many times a job(s) submitted to the IBM Q network can take a long time to process, e.g. jobs with many circuits and/or shots, or may have to wait in queue for other users.  In situations such as these, it is beneficial to have a way of monitoring the progress of a job, or several jobs at once.  As of Qiskit `0.6+` it is possible to monitor the status of a job in a Jupyter notebook, and also in a Python script (verision `0.7+`).

Lets see how to make use of these tools.

### Monitoring the status of a single job

Lets build a simple Bell circuit, submit it to a device, and then monitor its status.

In [3]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.measure(q, c);

Lets grab the least busy backend

In [4]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(IBMQ.backends(filters=lambda x: x.configuration().n_qubits <= 5 and 
              not x.configuration().simulator and x.status().operational==True))
backend.name()

'ibmqx4'

Monitor the job using `job_monitor` in blocking-mode (i.e. using the same thread as the Python interpretor)

In [5]:
job1 = execute(qc, backend)
job_monitor(job1)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

Monitor the job using `job_monitor` in async-mode (Jupyter notebooks only).  The job will be monitored in a separate thread, allowing you to continue to work in the notebook.

In [6]:
job2 = execute(qc, backend)
job_monitor(job2, monitor_async=True)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

### Monitoring many jobs simultaneously

Here we will monitor many jobs sent the the device.  It is if the jobs are stored in a list to make retrevial easier.

In [7]:
num_jobs = 5
my_jobs = []
for j in range(num_jobs):
    my_jobs.append(execute(qc, backend))
    job_monitor(my_jobs[j], monitor_async=True)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

### Changing the interval of status updating

By default, the interval at which the job status is checked is every two seconds.  However, the user is free to change this using the `interval` keyword argument in `job_monitor`

In [8]:
job3 = execute(qc, backend)
job_monitor(job3, interval=5)

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

## Backend Details

So far we have been executing our jobs on a backend, but we have explored the backends in any detail.  For example, we have found the least busy backend, but do not know if this is the best backend with respect to gate errors, topology etc.  It is possible to get detailed information for a single backend by calling `backend_monitor`:

In [9]:
backend_monitor(backend)

ibmqx4
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 14
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    url: None
    backend_name: ibmqx4
    n_registers: 1
    credits_required: True
    online_date: 2018-11-06T05:00:00+00:00
    max_experiments: 75
    description: 5 qubit device
    sample_name: raven
    coupling_map: [[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]]
    conditional: False
    memory: True
    max_shots: 8192
    open_pulse: False
    allow_q_object: True
    backend_version: 1.0.0

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.24985 GHz / 52.87796 µs / 45.91462 µs / 0.0 / 0.00077 / 0.00155 / 0.06025
    Q1 / 5.29578 GHz / 52.18911 µs / 19.45196 µs / 0.0 / 0.00197 / 0.00395 / 0.07425
    Q2 / 5.35325 GHz / 42.88025 µs / 29.48086 µs / 0.0 / 0.00163 / 0.00326

Or, if we are interested in a higher-level view of all the backends available to us, then we can use `backend_overview()`

In [10]:
backend_overview()

ibmq_20_tokyo               ibmq_16_melbourne            ibmqx4
-------------               -----------------            ------
Num. Qubits:  20            Num. Qubits:  14             Num. Qubits:  5
Pending Jobs: 0             Pending Jobs: 8              Pending Jobs: 14
Least busy:   True          Least busy:   False          Least busy:   False
Operational:  True          Operational:  True           Operational:  True
Avg. T1:      86.9          Avg. T1:      50.3           Avg. T1:      47.2
Avg. T2:      55.3          Avg. T2:      63.0           Avg. T2:      24.7





There are also Jupyter magic equivalents that give more detailed information.